In [5]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [6]:
data = pd.read_csv('F:\shivani\DL\DL_6\dataset\IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [8]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
english_stops = set(stopwords.words('english'))

In [10]:
def load_dataset():
    df = pd.read_csv('F:\shivani\DL\DL_6\dataset\IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\shiva\AppData\Local\Temp\ipykernel_6708\1841070064.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
47551    [i, thought, film, poor, effort, british, film...
42952    [since, many, users, already, explained, comme...
38116    [carson, daly, late, night, talk, show, host, ...
4997     [once, jet, li, brings, charismatic, presence,...
48290    [the, second, animated, movie, people, still, ...
                               ...                        
37880    [warned, movie, tells, lots, love, stories, wi...
34448    [excellent, film, dealing, life, old, man, loo...
25272    [mostly, uninvolving, biblical, mumbo, jumbo, ...
26535    [i, could, take, eyes, movie, showed, cable, t...
7989     [warning, post, may, contain, spoilers, i, got...
Name: review, Length: 40000, dtype: object 

27471    [it, genuine, shame, spin, tv, series, inspire...
4228     [a, letter, guys, i, tried, guys, i, really, t...
49912    [this, incredible, piece, drama, powerful, hit...
26448    [while, special, effects, technical, attribute...
41637    [johnathan, frakes, good, actor, directing, fa...
 

In [12]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [13]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    1   103     4 ...     0     0     0]
 [  141    37  6220 ... 26543 12410   594]
 [ 7370 15100   431 ...     0     0     0]
 ...
 [  552 12693  5909 ...     5  2010    12]
 [    1    27    97 ...     0     0     0]
 [ 1662  1017   110 ... 19323  3543    50]] 

Encoded X Test
 [[   7 1891  780 ...  171 1318    2]
 [  39 2948  354 ...    0    0    0]
 [   8  979  320 ...    0    0    0]
 ...
 [ 838  106  483 ...    0    0    0]
 [   1  164   11 ...    0    0    0]
 [3477  674 6260 ...    0    0    0]] 

Maximum review length:  130


In [14]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

c:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [16]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.5458 - loss: 0.6753
Epoch 1: accuracy improved from None to 0.61145, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 142ms/step - accuracy: 0.6115 - loss: 0.6433
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.6646 - loss: 0.6183
Epoch 2: accuracy improved from 0.61145 to 0.62498, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 44s 140ms/step - accuracy: 0.6250 - loss: 0.6406
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.6075 - loss: 0.6560
Epoch 3: accuracy improved from 0.62498 to 0.65167, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 151ms/step - accuracy: 0.6517 - loss: 0.6249
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.7405 - loss: 0.5671
Epoch 4: accuracy improved from 0.65167 to 0.73773, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 45s 143ms/step - accuracy: 0.7377 - loss: 0.5679
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.6468 - loss: 0.6170
Epoch 5: accuracy did not improve from 0.73773
313/313 ━━━━━━━━━━━━━━━━━━━━ 44s 140ms/step - accuracy: 0.6283 - loss: 0.6374


In [18]:
y_pred = model.predict(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 146ms/step
Correct Prediction: 0
Wrong Prediction: 10000
Accuracy: 0.0


In [19]:
loaded_model = load_model('models/LSTM.h5')

In [20]:
review = str(input('Movie Review: '))

In [22]:
# Pre-process input
import re
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [23]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [24]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
[[0.82953596]]


In [25]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
